Run locally on Vrael

In [1]:
import numpy as np
import scipy.stats as sps
from glob import glob
import uproot as ur
import ROOT as rt
import ROOT.RooFit as rf
from scipy.special import erf
from scipy.optimize import fsolve
from histo_utilities import create_TH1D, create_TH2D, std_color_list, SetMaxToMaxHist
from cebefo_style import Set_2D_colz_graphics
from particle import Particle

from analysis_utilities import drawOnCMSCanvas, extarct, extarct_multiple, createSel

from matplotlib import pyplot as plt

Welcome to JupyROOT 6.12/07


ImportError: No module named histo_utilities

In [ ]:
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

In [ ]:
from uproot_methods.classes import TLorentzVector as urTLV

In [ ]:
# tmu = ur.open('../data/cmsMC_private/BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_SoftQCD_PTFilter5_0p0-evtgen_HQET2_central_PU35_10-2-3_v0/B02DstMu_candidates.root')['outA']['Tevts']
tmu = ur.open('../data/cmsMC_private/BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_SoftQCD_PTFilter5_0p0-evtgen_HQET2_central_PU35_10-2-3_v0/B02DstMu_candidates.root')['outA']['Tevts']
print tmu.keys(), '\n\n'
branches = [b for b in tmu.keys() if b[:2] == 'MC']
for b in np.sort(branches):
    print b
    
dmu = {}
for b in branches:
    dmu[b] = tmu.array(b)

In [ ]:
def create_urTLV(d, k, m):
    px = d[k+'_pt'] * np.cos(d[k+'_phi'])
    py = d[k+'_pt'] * np.sin(d[k+'_phi'])
    pz = d[k+'_pt'] * np.sinh(d[k+'_eta']).astype(np.float64)
    
    e = np.sqrt(m**2 + np.square(d[k+'_P']))
    return urTLV.TLorentzVectorArray(px, py, pz, e)

In [ ]:
PB = create_urTLV(dmu, 'MC_B', 1e-3*Particle.from_string('B0').mass)
Pmu = create_urTLV(dmu, 'MC_mu', 1e-3*Particle.from_string('mu').mass)
PDst = create_urTLV(dmu, 'MC_Dst', 1e-3*Particle.from_string('D*-').mass)

Pv = Pmu + PDst

In [ ]:
plt.hist(PB.mass)
print np.mean(PB.mass)-1e-3*Particle.from_string('B0').mass, np.std(PB.mass)

Now get the B momentum form different approximations

## MC truth

In [ ]:
MCtruth = {}
MCtruth['B_pz'] = PB.z
MCtruth['B_p3'] = PB.p3

MCtruth['PB'] = PB

MCtruth['M2_miss'] = dmu['MC_M2_miss']
MCtruth['q2'] = dmu['MC_q2']

## Longitudinal Approx (a-la-LHCb)

In [ ]:
LongApp = {}
LongApp['B_pz'] = Pv.z * 1e-3*Particle.from_string('B0').mass / Pv.mass
LongApp['B_p3'] = PB.p3 * LongApp['B_pz'] / PB.p3.z

e = np.sqrt(LongApp['B_p3'].mag2 + (1e-3*Particle.from_string('B0').mass)**2)
LongApp['PB'] = urTLV.TLorentzVectorArray(LongApp['B_p3'].x, LongApp['B_p3'].y, LongApp['B_p3'].z, e)

LongApp['M2_miss'] = (LongApp['PB'] - Pv).mass2
LongApp['q2'] = (LongApp['PB'] - PDst).mass2

In [ ]:
h = create_TH2D(
            np.column_stack((np.abs(PB.eta), (PB.z-LongApp['B_pz'])/PB.z)),
            binning = [np.logspace(-2, np.log10(4), 15), np.linspace(-3, 3, 50)],
            axis_title = ['|#eta_{B}|', '|#DeltaP_{z}^{B}/P_{z}^{B}|', '']
           )

hpfy = h.ProfileX('hpfy', 1, -1, 's')
hpfy.SetLineColor(2)
c = drawOnCMSCanvas(CMS_lumi, [h, hpfy], ['colz', 'sameE1'])
c.SetLogx()

## Transverse Approx

In [ ]:
TranApp = {}
TranApp['B_pt'] = Pv.pt * 1e-3*Particle.from_string('B0').mass / Pv.mass
TranApp['B_p3'] = PB.p3 * TranApp['B_pt'] / PB.pt
TranApp['B_pz'] = TranApp['B_p3'].z

e = np.sqrt(TranApp['B_p3'].mag2 + (1e-3*Particle.from_string('B0').mass)**2)
TranApp['PB'] = urTLV.TLorentzVectorArray(TranApp['B_p3'].x, TranApp['B_p3'].y, TranApp['B_p3'].z, e)

TranApp['M2_miss'] = (TranApp['PB'] - Pv).mass2
TranApp['q2'] = (TranApp['PB'] - PDst).mass2

In [ ]:
h = create_TH2D(
            np.column_stack((np.abs(PB.eta), (PB.z-TranApp['B_pz'])/PB.z)),
            binning = [np.logspace(-2, np.log10(4), 15), np.linspace(-3, 3, 50)],
            axis_title = ['|#eta_{B}|', '|#DeltaP_{z}^{B}/P_{z}^{B}|', '']
           )

hpfy = h.ProfileX('hpfy', 1, -1, 's')
hpfy.SetLineColor(2)
c = drawOnCMSCanvas(CMS_lumi, [h, hpfy], ['colz', 'sameE1'])
c.SetLogx()

In [ ]:
np.min(TranApp['q2'])

## BoostBCM

In [ ]:
def equations(x, *a):
    g, pxst, pyst, pzst = x
    nx, ny, nz, e, px, py, pz, mv = a
    
    gb = np.sqrt(g*g-1)
    
    eqE = g*e - gb*nx*px - gb*ny*py - gb*nz*pz
    eqE -= np.sqrt(mv*mv + pxst*pxst + pyst*pyst + pzst*pzst)
    
    eqPx = -gb*nx*e 
    eqPx += px*( 1. + (g-1)*nx*nx )
    eqPx += py*(g-1)*nx*ny
    eqPx += pz*(g-1)*nx*nz
    eqPx -= pxst
    
    eqPy = -gb*ny*e
    eqPy += px*(g-1)*ny*nx
    eqPy += py*( 1. + (g-1)*ny*ny )
    eqPy += pz*(g-1)*ny*nz
    eqPy -= pyst
    
    eqPz = -gb*nz*e
    eqPz += px*(g-1)*nz*nx
    eqPz += py*(g-1)*nz*ny
    eqPz += pz*( 1. + (g-1)*nz*nz )
    eqPz -= pzst
    
    return (eqE, eqPx, eqPy, eqPz)

### Test

Get the arguments

In [ ]:
for i_evt in range(1):
    print '------- Event', i_evt, '--------'
    nx = PB.x[i_evt]/PB.p[i_evt]
    ny = PB.y[i_evt]/PB.p[i_evt]
    nz = PB.z[i_evt]/PB.p[i_evt]
    e = Pv.E[i_evt]
    px = Pv.x[i_evt]
    py = Pv.y[i_evt]
    pz = Pv.z[i_evt]
    mv = Pv.mass[i_evt]
    
    # Get the starting estimation from transverse approx
    g_init = TranApp['PB'].gamma[i_evt]
    P_vis_lv = rt.TLorentzVector()
    P_vis_lv.SetPxPyPzE(px, py, pz, e)
    P_vis_lv.Boost(-TranApp['PB'].boostp3[i_evt].x, -TranApp['PB'].boostp3[i_evt].y, -TranApp['PB'].boostp3[i_evt].z)
    
    pxst_init = P_vis_lv.Px()
    pyst_init = P_vis_lv.Py()
    pzst_init = P_vis_lv.Pz()
    
    print 'Initial status'
    print 'Init: {:.2f} {:.2f} {:.2f} {:.2f}'.format(g_init, pxst_init, pyst_init, pzst_init)
    r0, r1, r2, r3 =  equations((g_init, pxst_init, pyst_init, pzst_init), nx, ny, nz, e, px, py, pz, mv)
    print 'Loss val: {:.2e} {:.2e} {:.2e} {:.2e}'.format(r0, r1, r2, r3)
    print
    
    # Mimization step
    out = fsolve(
               equations, 
               x0=(g_init, pxst_init, pyst_init, pzst_init), 
               args = (nx, ny, nz, e, px, py, pz, mv)
              )
    g_opt, pxst_opt, pyst_opt, pzst_opt = out
    print 'Final status'
    print 'Init: {:.2f} {:.2f} {:.2f} {:.2f}'.format(g_opt, pxst_opt, pyst_opt, pzst_opt)
    r0, r1, r2, r3 =  equations((g_opt, pxst_opt, pyst_opt, pzst_opt), nx, ny, nz, e, px, py, pz, mv)
    print 'Loss val: {:.2e} {:.2e} {:.2e} {:.2e}'.format(r0, r1, r2, r3)
    print
    
    # Get the exact solution
    g_true = PB.gamma[i_evt]

    P_vis_lv = rt.TLorentzVector()
    P_vis_lv.SetPxPyPzE(Pv.x[i_evt], Pv.y[i_evt], pz, e)
    P_vis_lv.Boost(-PB.boostp3[i_evt].x, -PB.boostp3[i_evt].y, -PB.boostp3[i_evt].z)
    
    pxst_true = P_vis_lv.Px()
    pyst_true = P_vis_lv.Py()
    pzst_true = P_vis_lv.Pz()

    print 'True status'
    print 'Init: {:.2f} {:.2f} {:.2f} {:.2f}'.format(g_true, pxst_true, pyst_true, pzst_true)
    r0, r1, r2, r3 =  equations((g_true, pxst_true, pyst_true, pzst_true), nx, ny, nz, e, px, py, pz, mv)
    print 'Loss val: {:.2e} {:.2e} {:.2e} {:.2e}'.format(r0, r1, r2, r3)
    print '\n'

### For all events

In [ ]:
gamma = []
for i_evt in range(PB.z.shape[0]):
    nz = PB.z[i_evt]/PB.p[i_evt]
    nt = PB.pt[i_evt]/PB.p[i_evt]
    e = Pv.E[i_evt]
    pz = Pv.z[i_evt]
    pt = Pv.pt[i_evt]
    mv = Pv.mass[i_evt]
    
    # Get the starting estimation from transverse approx
    g_init = TranApp['PB'].gamma[i_evt]
    P_vis_lv = rt.TLorentzVector()
    P_vis_lv.SetPxPyPzE(Pv.x[i_evt], Pv.y[i_evt], pz, e)
    P_vis_lv.Boost(-TranApp['PB'].boostp3[i_evt].x, -TranApp['PB'].boostp3[i_evt].y, -TranApp['PB'].boostp3[i_evt].z)
    
    pzst_init = P_vis_lv.Px()
    ptst_init = P_vis_lv.Pt()
    
    # Mimization step
    out = fsolve(
               equations, 
               x0=(g_init, pzst_init, ptst_init), 
               args = (nz, nt, e, pz, pt, mv)
              )
    g_opt, pzst_opt, ptst_opt = out
    gamma.append(g_opt)

In [ ]:
gamma = np.array(gamma)

BoostBCM = {}

BoostBCM['gamma'] = gamma
BoostBCM['B_P'] = np.sqrt(np.square(gamma) - 1) * 1e-3*Particle.from_string('B0').mass

BoostBCM['B_p3'] = PB.p3 * BoostBCM['B_P'] / PB.p
BoostBCM['B_pt'] = np.hypot(BoostBCM['B_p3'].x, BoostBCM['B_p3'].y)
BoostBCM['B_pz'] = BoostBCM['B_p3'].z

e = np.sqrt(BoostBCM['B_p3'].mag2 + (1e-3*Particle.from_string('B0').mass)**2)
BoostBCM['PB'] = urTLV.TLorentzVectorArray(BoostBCM['B_p3'].x, BoostBCM['B_p3'].y, BoostBCM['B_p3'].z, e)

BoostBCM['M2_miss'] = (BoostBCM['PB'] - Pv).mass2
BoostBCM['q2'] = (BoostBCM['PB'] - PDst).mass2

# Variable comparison

In [ ]:
dic_list = {'MC Truth': MCtruth,
            'Long. approx': LongApp,
            'Trans. approx': TranApp,
            'Boost B cm': BoostBCM
           }

In [ ]:
def draw_var(vname, xtitle, binning, opt, leg_loc):
    h_l = []

    leg = rt.TLegend(leg_loc[0],leg_loc[1],leg_loc[2],leg_loc[3])
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    for n, d in dic_list.iteritems():
        h_aux = create_TH1D(
            d[vname], 
            axis_title=[xtitle, 'Events'], 
            binning=binning, opt=opt
        )
        h_aux.SetLineColor(std_color_list[len(h_l)])
        h_aux.Sumw2()
        leg.AddEntry(h_aux, n, 'l')
        h_l.append(h_aux)
    SetMaxToMaxHist(h_l)


    CMS_lumi.extraText = "     Simulation Preliminary"
    c = drawOnCMSCanvas(CMS_lumi, h_l, 'same')
    c.h_l = h_l
    c.leg = leg
    leg.Draw()
    return c

In [ ]:
c = draw_var('B_pz', 'p_{z}^{B} [GeV]', binning=[50, -150, 150], opt='underflowoverflow', leg_loc=[0.2,0.6,0.5,0.8])

In [ ]:
c = draw_var('q2', 'q^{2} [GeV^{2}]', binning=[30, -10, 12], opt='underflow', leg_loc=[0.2,0.6,0.5,0.8])

In [ ]:
c = draw_var('M2_miss', 'M^{2}_{miss} [GeV^{2}]', binning=[30, -3, 4], opt='underflowoverflow', leg_loc=[0.2,0.6,0.5,0.8])
c.SetLogy()